In [1]:
%matplotlib inline
import pandas as pd
#import matplotlib.pyplot as plt
import bqplot
from bqplot import pyplot as plt
from bqplot import *
import cartopy
import us
import ipywidgets
import matplotlib.gridspec as gridspec

<h1> Data Loading and Cleaning </h1>

In [2]:
#load the ufo file
colnames = ["date", "city", "state", "country", "shape",
          "duration_seconds", "duration_reported", "description",
          "report_date", "latitude", "longitude"]
ufo = pd.read_csv('/home/hbgrob/work/data-readonly/ufo-scrubbed-geocoded-time-standardized.csv', names = colnames, 
                  parse_dates=['date', 'report_date'])
ufo.head()

,date,city,state,country,shape,duration_seconds,duration_reported,description,report_date,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611


In [3]:
#load the alcohol per capita file
booze = pd.read_csv('cleaned_alcohol_data_abbr.csv')
#For this, we are just going to use the 'All beverages' column
booze = booze.drop(['Beer', 'Wine', 'Spirits','U.S. decile for all beverages'],axis=1)
booze.head()

,State or geographic area,Year,All beverages
0,AL,2015,2.00
1,AL,2014,1.99
2,AL,2013,1.99
3,AL,2012,2.00
4,AL,2011,1.97


In [4]:
'''this removes all the non-US cities from the dataset'''

ufo['country'].map(lambda x: x == 'us')
ufo_us = ufo[ufo['country'].map(lambda x: x == 'us')]
ufo_us['state'] = ufo_us['state'].apply(lambda x: x.upper())
ufo_us['year'] = ufo_us['date'].apply(lambda x: x.year)
ufo = ufo_us
ufo.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date,city,state,country,shape,duration_seconds,duration_reported,description,report_date,latitude,longitude,year
0,1949-10-10 20:30:00,san marcos,TX,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,1949
3,1956-10-10 21:00:00,edna,TX,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833,1956
4,1960-10-10 20:00:00,kaneohe,HI,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611,1960
5,1961-10-10 19:00:00,bristol,TN,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889,1961
7,1965-10-10 23:45:00,norwalk,CT,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,1999-10-02,41.117500,-73.408333,1965


In [5]:
#Because the date ranges are different, we must remove some rows outside the date range
booze_dateMax = booze['Year'].max()
booze_dateMin = booze['Year'].min()

print('the booze min = ' + str(booze_dateMin) +  ' the booze max = ' + str(booze_dateMax))
print('average = ' + str((booze_dateMax+booze_dateMin)//2) + '\n')

ufo_updatedYears = ufo[ufo['date'].map(lambda x: x.year >= booze_dateMin and x.year <= booze_dateMax)]

ufo_dateMax = int(ufo['date'].max().year)
ufo_dateMin = int(ufo['date'].min().year)

print('the ufo min is = ' + str(ufo_dateMin) + 'the ufo max is =' + str(ufo_dateMax))
print('average = ' + str((ufo_dateMax+ufo_dateMin)//2) + '\n')
#booze.head()

booze_updatedYears = booze[booze['Year'].map(lambda x: x >= ufo_dateMin and x <= ufo_dateMax)]

total_max = min(booze_dateMax, ufo_dateMax)
total_min = max(booze_dateMin, ufo_dateMin)
print("The date range is: " + str(total_min) + " - " + str(total_max))
print('average = ' + str((booze_dateMax+booze_dateMin)//2))
'''Since the closest census was in 1990, we will use that data'''

the booze min = 1977 the booze max = 2015
average = 1996

the ufo min is = 1910the ufo max is =2014
average = 1962

The date range is: 1977 - 2014
average = 1996


'Since the closest census was in 1990, we will use that data'

In [6]:
ufo = ufo_updatedYears
booze = booze_updatedYears

In [7]:
'''Populations'''
#1990 Census data found from:
#    https://www.census.gov/population/www/censusdata/pop1790-1990.html
pop = pd.read_csv('Population_PartII.csv')
pop.head()
ind = list(pop.columns)
pop_new = pop[ind[:2]] #only get the state and the 1990 data by state
pop = pop_new.iloc[1:52]
pop.head()

,State,1990
1,Alabama,"4,040,587"
2,Alaska,"550,043"
3,Arizona,"3,665,228"
4,Arkansas,"2,350,725"
5,California,"29,760,021"


In [8]:
# The percent of 14 year old's is below, found from the World Bank's website: 
#    https://data.worldbank.org/indicator/SP.POP.0014.TO.ZS?end=2016&start=1960&view=chart
#    This is needed because the alcohol data was collected from 14 year olds and older.
#    The percent is 21.6841780549198
percent_of_14 = (100 - 21.6841780549198)/100


#Going to find the percent of 14 year olds for each state
pop['1990'] = pop['1990'].apply(lambda x: int(x.replace(',',''))*percent_of_14)
pop.head()
pop_dict = pop.set_index('State').T.to_dict()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
#going to apply this dictionary to the booze dataframe
booze.head()

ind = booze.columns
temp = booze.values.tolist()
us_abbr_name = us.states.mapping('abbr', 'name')

#here is the multiplier
for item in temp:
    st = item[0]
    bpc = item[-1]
    state_full = us_abbr_name[st]
    bo = int(pop_dict[state_full]['1990']*bpc)
    item[-1]=bo
    #print(item)

booze_pop = pd.DataFrame(temp,columns = ind)
booze_pop.head()

,State or geographic area,Year,All beverages
0,AL,2014,6297193
1,AL,2013,6297193
2,AL,2012,6328837
3,AL,2011,6233905
4,AL,2010,6233905


In [10]:
state_list = ufo['state'].unique().tolist()
@ipywidgets.interact(state = state_list)
def make_plot(state = 'IL'):
    figx = []
    figy = []
    fig_layout = widgets.Layout(width='auto', height='auto')
    
    ####Top 
    
    ###
    # This is the interactive BQPLOT Map Plot - Aggregate of Sightings per state
    ###
    abbr_to_fits = us.states.mapping('abbr', 'fips')
    ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))

    fips_sightings_count = ufo.groupby("fips")["duration_seconds"].count()

    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                             'color': bqplot.ColorScale(colors=["lightblue", "darkblue"])},
                  'color': fips_sightings_count.to_dict()}

    mapTT = Tooltip(fields=['name', 'color',], labels=['State', 'Sightings'])
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'),
            **map_styles, tooltip= mapTT)
    map_title = 'UFO Sightings for US States from '+ str(total_min) + '-' + str(total_max)
    map_fig = bqplot.Figure(marks=[states_map], title=map_title,fig_margin={'top':30, 'bottom':30, 'left':0, 'right':0})
    figx.append(map_fig)
    
    
    
    
    ###
    # This is the plot that displays the sightings per year for the given state
    ###
    ufo_state_sightings_count = ufo[ufo['state'].apply(lambda x: x==state)]
    y_dataTop = ufo_state_sightings_count.groupby(['year'])['date'].count().tolist()
    x_dataTop = ufo['year'].unique().tolist()
    sc_x = LinearScale()
    sc_y = LinearScale()
    
    scatter = Scatter(x=x_dataTop, y=y_dataTop, scales={'x': sc_x, 'y': sc_y}, colors=['blue'])
    
    ax_x = Axis(label='Year', scale=sc_x)
    ax_y = Axis(label='Total Sightings', scale=sc_y, orientation='vertical', tick_format='0.2f')
    mapTT = Tooltip(fields=['name', x_dataTop,], labels=['State', 'Sightings'])
    year_count_title = 'Total Sighting for ' + state + ' per year'
    year_count = Figure(axes=[ax_x, ax_y], title = year_count_title, marks=[scatter], tooltip= mapTT)
    figx.append(year_count)
    figy.append(widgets.HBox(figx))
    figx = []
    
    
    ####Middle Row    
    
    ###
    # This is the interactive BQPLOT Map Plot - Aggregate Gallons of Alcohol Consumed per state
    ###
    
    abbr_to_fits = us.states.mapping('abbr', 'fips')
    booze['fips'] = booze['State or geographic area'].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))

    fips_booze_count = booze.groupby("fips")["All beverages"].sum()

    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                             'color': bqplot.ColorScale(colors=["lightblue", "darkblue"])},
                  'color': fips_booze_count.to_dict()}

    mapTT = Tooltip(fields=['name', 'color',], labels=['Year', 'Gallons of Alcohol Consumed'])
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'),
                **map_styles, tooltip= mapTT)
    map_title = 'Gallons of Alcohol Consumed for US States from '+ str(total_min) + '-' + str(total_max)
    map_booze_left_fig = bqplot.Figure(marks=[states_map], title=map_title,fig_margin={'top':30, 'bottom':30, 'left':0, 'right':0})
    figx.append(map_booze_left_fig)
    
    
    
    
    ###
    # This is the plot that displays the total duration of the sightings per year for the given state
    ###
    ufo_state_sightings_duration = ufo[ufo['state'].apply(lambda x: x==state)]
    y_dataMid = ufo_state_sightings_duration.groupby(['year'])['duration_seconds'].sum().tolist()
    x_dataMid = ufo['year'].unique().tolist()
    sc_x = LinearScale()
    sc_y = LinearScale()
    
    scatter = Scatter(x=x_dataMid, y=y_dataMid, scales={'x': sc_x, 'y': sc_y}, colors=['blue'])
    ax_x = Axis(label='Year', scale=sc_x)
    ax_y = Axis(label='Total Duration of Sightings', scale=sc_y, orientation='vertical', tick_format='0.2f')
    year_duration_title = 'Total Duration of Sightings for ' + state
    year_duration = Figure(axes=[ax_x, ax_y], marks=[scatter], title = year_duration_title)
    figx.append(year_duration)
    
    figy.append(widgets.HBox(figx))
    figx=[]
    
    
    ###
    #  This represents the number of gallons consumed per sighting. Each dot represents a different year
    ###
    gallons_consumed = booze[booze['State or geographic area'].apply(lambda x: x==state)]
    y_dataLow = ufo_state_sightings_count.groupby(['year'])['date'].count().tolist()
    x_dataLow = gallons_consumed.groupby('Year')['All beverages'].sum().tolist()
    sc_x = LinearScale()
    sc_y = LinearScale()#min = min(y_dataLow), max = max(y_dataLow))
    scatter = Scatter(x=x_dataLow, y=y_dataLow, scales={'x': sc_x, 'y': sc_y}, colors=['red'])
    ax_x = Axis(label='Gallons Consumed', scale=sc_x)
    ax_y = Axis(label='Number of Yearly Sightings', scale=sc_y, orientation='vertical', tick_format='0.2f')
    year_gallon_title = 'Gallons Consumed per Sighting for ' + state
    year_gallon = Figure(axes=[ax_x, ax_y], marks=[scatter], title = year_gallon_title)
    figx.append(year_gallon)
    
    
    #ax4.yaxis.tick_right()
    #figx.append(map_fig)
    figy.append(widgets.HBox(figx))
    
    display(widgets.VBox(figy, align_content='stretch'))
    

interactive(children=(Dropdown(description='state', index=15, options=('TX', 'KY', 'NY', 'CA', 'CO', 'NH', 'MI', 'WI', 'ME', 'WA', 'GA', 'NC', 'PA', 'FL', 'TN', 'IL', 'AR', 'MO', 'SC', 'OH', 'IN', 'AZ', 'MN', 'NV', 'KS', 'NE', 'OR', 'IA', 'VA', 'ID', 'NM', 'NJ', 'WV', 'OK', 'CT', 'RI', 'AL', 'VT', 'LA', 'PR', 'AK', 'MS', 'MA', 'HI', 'UT', 'MD', 'MT', 'WY', 'SD', 'DE', 'ND', 'DC'), value='IL'), Output()), _dom_classes=('widget-interact',))